# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "demo_human_or_worm"
TOKENIZER = "character"


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_human_or_worm 0 character 0


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

/Users/katarina/Documents/repositories/GitHub/GLP/venv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Done.
Unzipping...

Done.


## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 9
{'N': 7, '<eos>': 6, 'G': 4, '<pad>': 8, '<unk>': 0, '<bos>': 1, 'T': 3, 'C': 5, 'A': 2}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  202


not all sequences are of the same length


In [9]:
tokenizer(train_dset[1][0])

['<bos>',
 'T',
 'T',
 'A',
 'A',
 'A',
 'C',
 'T',
 'T',
 'T',
 'A',
 'A',
 'A',
 'T',
 'T',
 'T',
 'T',
 'C',
 'C',
 'C',
 'A',
 'A',
 'A',
 'C',
 'T',
 'A',
 'C',
 'A',
 'G',
 'G',
 'A',
 'A',
 'C',
 'C',
 'C',
 'T',
 'T',
 'T',
 'G',
 'T',
 'T',
 'G',
 'A',
 'A',
 'T',
 'T',
 'T',
 'T',
 'A',
 'A',
 'C',
 'A',
 'G',
 'A',
 'T',
 'T',
 'T',
 'T',
 'T',
 'G',
 'A',
 'G',
 'C',
 'A',
 'C',
 'G',
 'A',
 'A',
 'T',
 'A',
 'C',
 'T',
 'A',
 'C',
 'A',
 'T',
 'T',
 'T',
 'T',
 'A',
 'A',
 'G',
 'A',
 'C',
 'A',
 'A',
 'T',
 'T',
 'T',
 'C',
 'C',
 'T',
 'C',
 'A',
 'T',
 'T',
 'C',
 'A',
 'T',
 'G',
 'T',
 'T',
 'A',
 'C',
 'T',
 'T',
 'C',
 'T',
 'T',
 'A',
 'T',
 'T',
 'T',
 'A',
 'A',
 'A',
 'G',
 'A',
 'A',
 'A',
 'T',
 'T',
 'T',
 'A',
 'T',
 'C',
 'A',
 'G',
 'A',
 'A',
 'A',
 'T',
 'T',
 'T',
 'A',
 'T',
 'T',
 'T',
 'A',
 'T',
 'T',
 'G',
 'A',
 'G',
 'G',
 'A',
 'C',
 'T',
 'A',
 'T',
 'G',
 'T',
 'A',
 'A',
 'T',
 'A',
 'A',
 'A',
 'C',
 'A',
 'T',
 'T',
 'T',
 'T',
 'T',
 'T',


## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 90.8%, Avg loss: 0.545845 

Epoch 1


Train metrics: 
 Accuracy: 91.8%, Avg loss: 0.539735 

Epoch 2


Train metrics: 
 Accuracy: 91.1%, Avg loss: 0.541308 

Epoch 3


Train metrics: 
 Accuracy: 92.5%, Avg loss: 0.536730 

Epoch 4


Train metrics: 
 Accuracy: 92.7%, Avg loss: 0.538045 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  11399.898065328598 ; fp  805.7902834847488
recall  0.9119918452262878 ; precision  0.9339823973497497
num_batches 782
correct 23100
size 25000
Test metrics: 
 Accuracy: 0.924000, F1 score: 0.922856, Avg loss: 0.539537 



(0.924, 0.9228561377749391)